In [1]:
#Model: LSTM
#Word Embedding: Custom
#Dataset: 3
#based on: http://www.insightsbot.com/blog/1wAqZg/keras-lstm-example-sequence-binary-classification
#https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py

import os

def GetTextFilePathsInDirectory(directory):
    files = []
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            filePath = os.path.join(directory, file)
            files.append(filePath)
    return files

def GetLinesFromTextFile(filePath):
    with open(filePath,"r", encoding="utf-8") as f:
        lines = [line.strip() for line in f]
    return lines

In [2]:
positive_files = GetTextFilePathsInDirectory("./dataset3/positive")
negative_files = GetTextFilePathsInDirectory("./dataset3/negative")

reviews_positive = []
for i in range(0,1620):
    reviews_positive.extend(GetLinesFromTextFile(positive_files[i]))
    
reviews_negative = []
for i in range(0,1620):
    reviews_negative.extend(GetLinesFromTextFile(negative_files[i]))

In [4]:
import nltk
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])$&%=#•~@£©®→°€™›♥←×§″′Â█½à…“★”–●â►¢²¬░¶↑±¿▾¦║―¥▓▒¼⊕▼▪†■▀¨▄♫☆é♦¤▲è¾Ã∞∙↓、,│»♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡√")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    default_stop_words = nltk.corpus.stopwords.words('german')
    stopwords = set(default_stop_words)
    
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    reviews = [RemoveStopWords(line,stopwords) for line in reviews]
    
    return reviews

def RemoveStopWords(line, stopwords):
    words = []
    for word in line.split(" "):
        word = word.strip()
        if word not in stopwords and word != "" and word != "&":
            words.append(word)

    return " ".join(words)

 

In [5]:
reviews_positive = preprocess_reviews(reviews_positive)
reviews_negative = preprocess_reviews(reviews_negative)

In [6]:
import numpy as np
Reviews_Labeled = list(zip(reviews_positive, np.ones(len(reviews_positive))))
Reviews_Labeled.extend(list(zip(reviews_negative, np.zeros(len(reviews_negative)))))

In [7]:
from collections import Counter
import json
class Vocabulary:
    
    def __init__(self, vocabulary, wordFrequencyFilePath):
        self.vocabulary = vocabulary
        self.WORD_FREQUENCY_FILE_FULL_PATH = wordFrequencyFilePath
        self.input_word_index = {}
        self.reverse_input_word_index = {}
        
        self.MaxSentenceLength = None
        
    def PrepareVocabulary(self,reviews):
        self._prepare_Word_Frequency_Count_File(reviews)
        self._create_Vocab_Indexes()
        
        self.MaxSentenceLength = max([len(txt.split(" ")) for txt in reviews])
      
    def Get_Top_Words(self, number_words = None):
        if number_words == None:
            number_words = self.vocabulary
        
        chars = json.loads(open(self.WORD_FREQUENCY_FILE_FULL_PATH).read())
        counter = Counter(chars)
        most_popular_words = {key for key, _value in counter.most_common(number_words)}
        return most_popular_words
    
    def _prepare_Word_Frequency_Count_File(self,reviews):
        counter = Counter()    
        for s in reviews:
            counter.update(s.split(" "))
            
        with open(self.WORD_FREQUENCY_FILE_FULL_PATH, 'w') as output_file:
            output_file.write(json.dumps(counter))
                 
    def _create_Vocab_Indexes(self):
        INPUT_WORDS = self.Get_Top_Words(self.vocabulary)

        for i, word in enumerate(INPUT_WORDS):
            self.input_word_index[word] = i
        
        for word, i in self.input_word_index.items():
            self.reverse_input_word_index[i] = word
       
    def TransformSentencesToId(self, sentences):
        vectors = []
        for r in sentences:
            words = r.split(" ")
            vector = np.zeros(len(words))

            for t, word in enumerate(words):
                if word in self.input_word_index:
                    vector[t] = self.input_word_index[word]
                else:
                    pass
                
            vectors.append(vector)
            
        return vectors
    
    def ReverseTransformSentencesToId(self, sentences):
        vectors = []
        for r in sentences:
            words = r.split(" ")
            vector = np.zeros(len(words))

            for t, word in enumerate(words):
                if word in self.input_word_index:
                    vector[t] = self.input_word_index[word]
                else:
                    pass
                    #vector[t] = 2 #unk
            vectors.append(vector)
            
        return vectors

In [8]:
TOP_WORDS = 500
vocab = Vocabulary(TOP_WORDS,"analysis11.vocab")

reviews_text = [line[0] for line in Reviews_Labeled]
vocab.PrepareVocabulary(reviews_text)

In [9]:
reviews,labels=zip(*Reviews_Labeled)
reviews_int = vocab.TransformSentencesToId(reviews)

Reviews_Labeled_Int = list(zip(reviews_int,labels))

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(Reviews_Labeled_Int, test_size=0.2)

In [11]:
X_train, y_train = list(zip(*train))
X_test, y_test = list(zip(*test))

y_train = np.array(y_train)
y_test = np.array(y_test)

In [12]:
from keras.preprocessing import sequence 
max_review_length = 500 
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length) 
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length) 

Using TensorFlow backend.


In [13]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM
import keras 

embedding_vector_length = 32
model = Sequential()
model.add(Embedding(TOP_WORDS, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           16000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 500, 100)          53200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 149,701
Trainable params: 149,701
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Instructions for updating:
Use tf.cast instead.
Train on 58620 samples, validate on 14655 samples
Epoch 1/10
58620/58620 [==============================] - 1651s 28ms/step - loss: 0.5454 - acc: 0.7509 - val_loss: 0.5363 - val_acc: 0.7485
Epoch 2/10
58620/58620 [==============================] - 1146s 20ms/step - loss: 0.5341 - acc: 0.7548 - val_loss: 0.5742 - val_acc: 0.7387
Epoch 3/10
58620/58620 [==============================] - 1144s 20ms/step - loss: 0.5635 - acc: 0.7441 - val_loss: 0.5538 - val_acc: 0.7387
Epoch 4/10
58620/58620 [==============================] - 1143s 19ms/step - loss: 0.5559 - acc: 0.7441 - val_loss: 0.5475 - val_acc: 0.7387
Epoch 5/10
58620/58620 [==============================] - 1130s 19ms/step - loss: 0.5235 - acc: 0.7598 - val_loss: 0.5298 - val_acc: 0.7636
Epoch 6/10
58620/58620 [==============================] - 653s 11ms/step - loss: 0.5067 - acc: 0.7750 - val_loss: 0.5179 - val_acc: 0.7678
Epoch 7/10
58620/58620 [==============================] - 654s 